# Exercício


Neste exercício, tentaremos identificar diferentes `perfis de clientes` de um supermercado. O propósito será de identificar clusters semelhantes para que sejam criadas estratégias de marketing e vendas mais adequadas para cada grupo, incentivando a fidelidade dos clientes e potencializando as vendas.

# Clusterização

### Atividade 1: Crie um dataframe a partir do banco de dados `customer_supermarket`.

O dataset está disponível em [formato .csv](https://drive.google.com/file/d/1L3ohl-YgU5SPcUS2LihG29Kj9XPE6er9/view?usp=share_link) ou em [formato .xlsx](https://docs.google.com/spreadsheets/d/18Cl-4fdjVAV-mKD74zRFpQ9esz_nU7fP/edit?usp=share_link&ouid=111649208388214484177&rtpof=true&sd=true)


Abaixo encontra-se o dicionário de dados:


**Informações Pessoais**

`ID`: Identificador único do cliente

`Year_Birth`: Ano de nascimento do cliente

`Education`: Nível de escolaridade do cliente

`Marital_Status`: Estado civil do cliente

`Income`: Renda familiar anual do cliente

`Kidhome`: Número de filhos na casa do cliente

`Teenhome`: Número de adolescentes na casa do cliente

`Dt_Customer`: Data de cadastro do cliente

`Days_Registered`: Quantidade de Dias desde o cadastro

`Recency`: Número de dias desde a última compra do cliente

`Complain`: 1 se o cliente reclamou nos últimos 2 anos, 0 caso contrário



**Produtos**

`MntWines`: Valor gasto em vinho nos últimos 2 anos

`MntFruits`: Valor gasto em frutas nos últimos 2 anos

`MntMeatProducts`: Valor gasto em carnes nos últimos 2 anos

`MntFishProducts`: Valor gasto em pescados nos últimos 2 anos

`MntSweetProducts`: Valor gasto em doces nos últimos 2 anos

`MntGoldProds`: Valor gasto em ouro nos últimos 2 anos



**Promoções**

`NumDealsPurchases`: Número de compras feitas com desconto

`AcceptedCmp1`: 1 se o cliente aceitou a oferta na 1ª campanha, 0 caso contrário

`AcceptedCmp2`: 1 se o cliente aceitou a oferta na 2ª campanha, 0 caso contrário

`AcceptedCmp3`: 1 se o cliente aceitou a oferta na 3ª campanha, 0 caso contrário

`AcceptedCmp4`: 1 se o cliente aceitou a oferta na 4ª campanha, 0 caso contrário

`AcceptedCmp5`: 1 se o cliente aceitou a oferta na 5ª campanha, 0 caso contrário

`Response`: 1 se o cliente aceitou a oferta na última campanha, 0 caso contrário



**Meios**

`NumWebPurchases`: Número de compras realizadas pelo site da empresa

`NumCatalogPurchases`: Número de compras feitas usando um catálogo

`NumStorePurchases`: Número de compras feitas diretamente nas lojas

`NumWebVisitsMonth`: Número de visitas ao site da empresa no último mês


Obs: este dataset foi `adaptado` a partir do original encontrado no [kaggle](https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis).

In [27]:
import pandas as pd

import numpy as np

# Importando a biblioteca warnings para ignorar avisos durante a execução do código
import warnings
warnings.filterwarnings("ignore")

In [17]:
# Lendo o arquivo
df = pd.read_excel('customer_supermarket.xlsx')

df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Days_Registered,Recency,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1965,Graduation,Single,58138.0,0,0,2020-09-04,849,58,...,7,0,0,0,0,0,0,3,11,1
1,2174,1962,Graduation,Single,46344.0,1,1,2022-03-08,299,38,...,5,0,0,0,0,0,0,3,11,0
2,4141,1973,Graduation,Together,71613.0,0,0,2021-08-21,498,26,...,4,0,0,0,0,0,0,3,11,0
3,6182,1992,Graduation,Together,26646.0,1,0,2022-02-10,325,26,...,6,0,0,0,0,0,0,3,11,0
4,5324,1989,PhD,Married,58293.0,1,0,2022-01-19,347,94,...,5,0,0,0,0,0,0,3,11,0


### Atividade 2: Faça o tratamento dos Dados.
Dica: verifique `dados ausentes` e `outliers` e, se necessário, remova as linhas.

In [18]:
# Verificar dados ausentes
print("Dados ausentes por coluna:")
print(df.isnull().sum())

# Dropar dados ausentes
df.dropna(inplace=True)
df.isnull().sum()

Dados ausentes por coluna:
ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Days_Registered         0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64


ID                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Days_Registered        0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Z_CostContact          0
Z_Revenue              0
Response               0
dtype: int64

In [19]:
# Selecionando apenas colunas numéricas (excluindo colunas de data)
numerical_df = df.select_dtypes(include=[np.number])

# Verificar outliers usando o método IQR Score
Q1 = numerical_df.quantile(0.25)
Q3 = numerical_df.quantile(0.75)
IQR = Q3 - Q1
print("\nOutliers:")
print(((numerical_df < (Q1 - 1.5 * IQR)) | (numerical_df > (Q3 + 1.5 * IQR))).sum())


Outliers:
ID                       0
Year_Birth               3
Income                   8
Kidhome                  0
Teenhome                 0
Days_Registered          0
Recency                  0
MntWines                35
MntFruits              246
MntMeatProducts        174
MntFishProducts        222
MntSweetProducts       246
MntGoldProds           205
NumDealsPurchases       84
NumWebPurchases          3
NumCatalogPurchases     23
NumStorePurchases        0
NumWebVisitsMonth        8
AcceptedCmp3           163
AcceptedCmp4           164
AcceptedCmp5           162
AcceptedCmp1           142
AcceptedCmp2            30
Complain                21
Z_CostContact            0
Z_Revenue                0
Response               333
dtype: int64


In [21]:
# Verificar outliers usando o método Z-Score
outliers_count = {}

threshold = 3  # Definir o número de desvios padrão para considerar como outliers

for column in numerical_df.columns:
    mean = numerical_df[column].mean()
    std = numerical_df[column].std()
    lower_threshold = mean - threshold * std
    upper_threshold = mean + threshold * std
    outliers = numerical_df[(numerical_df[column] < lower_threshold) | (numerical_df[column] > upper_threshold)]
    outliers_count[column] = outliers.shape[0]

outliers_df = pd.DataFrame(outliers_count.items(), columns=['Coluna', 'Contagem'])
print(outliers_df)

                 Coluna  Contagem
0                    ID         0
1            Year_Birth         3
2                Income         8
3               Kidhome         0
4              Teenhome         0
5       Days_Registered         0
6               Recency         0
7              MntWines        15
8             MntFruits        64
9       MntMeatProducts        39
10      MntFishProducts        58
11     MntSweetProducts        61
12         MntGoldProds        46
13    NumDealsPurchases        31
14      NumWebPurchases         3
15  NumCatalogPurchases         4
16    NumStorePurchases         0
17    NumWebVisitsMonth         9
18         AcceptedCmp3       163
19         AcceptedCmp4       164
20         AcceptedCmp5       162
21         AcceptedCmp1       142
22         AcceptedCmp2        30
23             Complain        21
24        Z_CostContact         0
25            Z_Revenue         0
26             Response         0


In [22]:
outliers_count = {}
iqr_multiplier = 1.5  # Multiplicador do IQR para definir os limites de outliers
zscore_threshold = 3  # Limite de Z-score para definir outliers

for column in numerical_df.columns:
    # Contagem de outliers usando o método do IQR
    q1 = numerical_df[column].quantile(0.25)
    q3 = numerical_df[column].quantile(0.75)
    iqr = q3 - q1
    lower_threshold_iqr = q1 - iqr_multiplier * iqr
    upper_threshold_iqr = q3 + iqr_multiplier * iqr
    outliers_iqr = numerical_df[(numerical_df[column] < lower_threshold_iqr) | (numerical_df[column] > upper_threshold_iqr)]
    
    # Contagem de outliers usando o método do Z-score
    mean = numerical_df[column].mean()
    std = numerical_df[column].std()
    lower_threshold_zscore = mean - zscore_threshold * std
    upper_threshold_zscore = mean + zscore_threshold * std
    outliers_zscore = numerical_df[(numerical_df[column] < lower_threshold_zscore) | (numerical_df[column] > upper_threshold_zscore)]
    
    outliers_count[column] = {'IQR': outliers_iqr.shape[0], 'Z-score': outliers_zscore.shape[0]}

outliers_df = pd.DataFrame(outliers_count).transpose()
print(outliers_df)


                     IQR  Z-score
ID                     0        0
Year_Birth             3        3
Income                 8        8
Kidhome                0        0
Teenhome               0        0
Days_Registered        0        0
Recency                0        0
MntWines              35       15
MntFruits            246       64
MntMeatProducts      174       39
MntFishProducts      222       58
MntSweetProducts     246       61
MntGoldProds         205       46
NumDealsPurchases     84       31
NumWebPurchases        3        3
NumCatalogPurchases   23        4
NumStorePurchases      0        0
NumWebVisitsMonth      8        9
AcceptedCmp3         163      163
AcceptedCmp4         164      164
AcceptedCmp5         162      162
AcceptedCmp1         142      142
AcceptedCmp2          30       30
Complain              21       21
Z_CostContact          0        0
Z_Revenue              0        0
Response             333        0


In [40]:
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler, RobustScaler

# Criar uma instância do StandardScaler
scaler = RobustScaler()

alpha = 0.05  # Definir o nível de significância desejado

outliers_count = {}

for column in numerical_df.columns:
    # Contagem de outliers usando o método do IQR
    q1 = numerical_df[column].quantile(0.25)
    q3 = numerical_df[column].quantile(0.75)
    iqr = q3 - q1
    lower_threshold_iqr = q1 - iqr_multiplier * iqr
    upper_threshold_iqr = q3 + iqr_multiplier * iqr
    outliers_iqr = numerical_df[(numerical_df[column] < lower_threshold_iqr) | (numerical_df[column] > upper_threshold_iqr)]
    
    # Contagem de outliers usando o método do Z-score
    mean = numerical_df[column].mean()
    std = numerical_df[column].std()
    lower_threshold_zscore = mean - zscore_threshold * std
    upper_threshold_zscore = mean + zscore_threshold * std
    outliers_zscore = numerical_df[(numerical_df[column] < lower_threshold_zscore) | (numerical_df[column] > upper_threshold_zscore)]
    

    # Padronizar os dados
    dados_padronizados = scaler.fit_transform(numerical_df)

    # Criar um novo DataFrame com os dados padronizados
    df_padronizado = pd.DataFrame(dados_padronizados, columns=numerical_df.columns)

    # Contagem de outliers usando o método Isolation Forest
    isolation_forest = IsolationForest(contamination=alpha)
    isolation_forest.fit(df_padronizado[[column]])
    outliers_isolation_forest = isolation_forest.predict(df_padronizado[[column]])
    outliers_isolation_forest = numerical_df[outliers_isolation_forest == -1]

    # Apply DBSCAN to cluster the data and find outliers
    dbscan = DBSCAN(eps=0.2, min_samples=5)
    dbscan.fit(df_padronizado[[column]])
    outliers_DBSCAN = dbscan.labels_
    #outliers_DBSCAN = dbscan.predict(numerical_df[[column]])
    outliers_DBSCAN = numerical_df[outliers_DBSCAN == -1]
    
    outliers_count[column] = {'IQR': outliers_iqr.shape[0], 'Z-score': outliers_zscore.shape[0],
                               'Isolation Forest': outliers_isolation_forest.shape[0], 'DBSCAN': outliers_DBSCAN.shape[0]}

outliers_df = pd.DataFrame(outliers_count).transpose()
print(outliers_df)

                     IQR  Z-score  Isolation Forest  DBSCAN
ID                     0        0               111       0
Year_Birth             3        3               101       3
Income                 8        8               110       2
Kidhome                0        0                46       0
Teenhome               0        0                51       0
Days_Registered        0        0               110       0
Recency                0        0               111       0
MntWines              35       15               110       0
MntFruits            246       64               111       0
MntMeatProducts      174       39               111       4
MntFishProducts      222       58               105       0
MntSweetProducts     246       61               110       1
MntGoldProds         205       46               111       3
NumDealsPurchases     84       31                89       6
NumWebPurchases        3        3                95       3
NumCatalogPurchases   23        4       

In [ ]:
# Dropar outliers
numerical_df = numerical_df[~((numerical_df < (Q1 - 1.5 * IQR)) | (numerical_df > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
# seu código

### Atividade 3: Crie uma nova coluna chamada "MntTotal" que é a soma das colunas "MntFruits", "MntMeatProducts", "MntSweetProducts", "MntWines" e "MntGoldProds"

In [ ]:
# seu código

### Atividade 4: Crie um DataFrame apenas com as colunas `Recency`, `Days_Registered`, `MntTotal` e `Income` e use-o para fazer o agrupamento.

In [ ]:
# seu código

### Atividade 5: Padronize os dados.

In [ ]:
# seu código

### Atividade 6: Faça um agrupamento hierárquico e plote o dendograma.

In [ ]:
# seu código

In [ ]:
# seu código

### Atividade 7: Encontre a quantidade que parece adequada para o número de grupos.
Dicas:
* Faça a análise a partir do dendrograma acima, escolha uma quantidade de clusters que parecer adequada e crie uma coluna com o nome `grupos` no dataframe.
* Adicione a coluna `grupos` ao dataframe contendo todas as variáveis (e não somente aquelas utilizadas para os agrupamentos), assim você poderá fazer uma análise mais ampla dos consumidores.
* Para o propósito deste exercício, utilize um `máximo` de 4 grupos para permitir uma boa visualização nas etapas a seguir.

In [ ]:
# seu código

### Atividade 8: Crie Gráficos para visualizar os Grupos.

#### Atividade 8.1: Crie Visualizações em Boxplot
Dica: utilize o Seaborn para criar boxplots com os clusters selecionados.

In [ ]:
# seu código

In [ ]:
# seu código

In [ ]:
# seu código

In [ ]:
# seu código

#### Atividade 8.2: Crie Histogramas para visualizar os Grupos
Dica: utilize o Seaborn para plotar histogramas dos clusters selecionados.

In [ ]:
# seu código

In [ ]:
# seu código

In [ ]:
# seu código

In [ ]:
# seu código

In [ ]:
# seu código

In [ ]:
# seu código

### Atividade 9: Faça uma análise de cada grupo a fim de criar uma descrição adequada dele (persona).

Descrição dos Grupos:

* O grupo 1 é composto por pessoas que [...].

* O grupo 2 é composto por pessoas que [...].

* O grupo (n) é composto por pessoas que [...].

### Para refletir: 


####Quais outras variáveis poderíamos combinar ou processar para obter análises mais precisas?

Exemplo: e se comparássemos o valor gasto com a faixa de salário para entender esses valores percentualmente?

####Como poderíamos usar uma variável categórica para realizar agrupamento?

Dica:  Pesquise sobre One Hot Encoding